1. Se realizan las importaciones necesarias para la manipulación de los datos y construcción de ambos modelos a implementar

In [10]:
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, RobustScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

In [11]:
path = '../resources/Datos_Etapa_1_csv.csv'
with open(path, 'r') as file:
    content = file.read()

content = content.replace('"', '')  # Eliminar las comillas dobles

with open(path, 'w') as file:
    file.write(content)

In [12]:
#Carga y verificación del CSV 
data_original = pd.read_csv(path, sep=',')
data_copy = data_original.copy() #---- variable para manejo de transformación get_dummies
data_copy2 = data_original.copy() #--- variable para manejo de transformación .cat.codes
data_copy.head()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


In [13]:
print(data_copy.shape)
#----------------------------------------------------------------------------------transformación .cat.codes
#data_copy2['season'] = data_copy['season'].astype('category').cat.codes
#data_copy2['weathersit'] = data_copy['weathersit'].astype('category').cat.codes
#data_copy2['time_of_day'] = data_copy['time_of_day'].astype('category').cat.codes
#encoded_data = data_copy2
#---------------------------------------------------------------------------------Transformación get_dummies
categorical_columns = ['weekday','season', 'weathersit', 'time_of_day']
encoded_data = pd.get_dummies(data_copy, columns=categorical_columns)
categorical_encoded_columns = [col for col in encoded_data.columns if col not in data_copy.columns.difference(categorical_columns)]
encoded_data[categorical_encoded_columns] = encoded_data[categorical_encoded_columns].astype(int)   
#--------------------------------------------------------------------------------------------------------------------------
data_copy = encoded_data
data_copy.head()    

(17379, 9)


,temp,atemp,hum,windspeed,cnt,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,...,season_Spring,season_Summer,season_Winter,weathersit_Clear,weathersit_Heavy Rain,weathersit_Light Rain,weathersit_Mist,time_of_day_Evening,time_of_day_Morning,time_of_day_Night
0,3.28,3.0014,0.81,0.0,16,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
1,2.34,1.9982,0.80,0.0,40,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
2,2.34,1.9982,0.80,0.0,32,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,3.28,3.0014,0.75,0.0,13,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
4,3.28,3.0014,0.75,0.0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1


In [14]:
data_copy.isna().sum()

temp                     0
atemp                    0
hum                      0
windspeed                0
cnt                      0
weekday_0                0
weekday_1                0
weekday_2                0
weekday_3                0
weekday_4                0
weekday_5                0
weekday_6                0
season_Fall              0
season_Spring            0
season_Summer            0
season_Winter            0
weathersit_Clear         0
weathersit_Heavy Rain    0
weathersit_Light Rain    0
weathersit_Mist          0
time_of_day_Evening      0
time_of_day_Morning      0
time_of_day_Night        0
dtype: int64

In [15]:
#verificar registros duplicados 
data_copy.duplicated().sum()

np.int64(42)

In [16]:
#eliminar registros replicados
data_copy = data_copy.drop_duplicates()
print(data_copy.shape)

(17337, 23)


In [17]:
#data_copy = data_copy.drop(['windspeed'], axis = 1)
#data_copy = data_copy.drop(['weekday_2'],axis = 1)
#data_copy = data_copy.drop(['weekday_3'],axis = 1)
#data_copy = data_copy.drop(['weekday_4'],axis = 1)
#data_copy = data_copy.drop(['weathersit_Heavy Rain'],axis = 1)
#data_copy = data_copy.drop(['weathersit_Mist'],axis = 1)
#data_copy = data_copy.drop(['time_of_day_Morning'],axis = 1)
#data_copy.head()

In [18]:
#PRUEBA--------------------
x_pol = data_copy.drop(['cnt'], axis=1)
y_pol = data_copy['cnt']
x_train_pol, x_test_pol, y_train_pol, y_test_pol = train_test_split(x_pol, y_pol, test_size=0.2, random_state=77)
#BUSQUEDA DE HIPER PARAMETRO
polynomial_regression = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(),
    Lasso(max_iter= 2000)
)
kfold = KFold(n_splits=10, shuffle=True, random_state=77)
#espacio de busqueda 
valores_busqueda = [2, 3]
param_grid_pol = {'polynomialfeatures__degree': valores_busqueda,
                  'lasso__alpha': [1, 2, 3, 4, 5]}
grid_pol = GridSearchCV(polynomial_regression, param_grid_pol, cv=kfold, n_jobs=1, scoring='neg_mean_squared_error')
grid_pol.fit(x_train_pol, y_train_pol)

print("mejor parametro polinomial: ", grid_pol.best_params_)
mejor_modelo_pol = grid_pol.best_estimator_
#prediciones
y_pred_pol = mejor_modelo_pol.predict(x_test_pol)
#manejo de errores
rmse_pol = root_mean_squared_error(y_test_pol, y_pred_pol)
mae_pol = mean_absolute_error(y_test_pol, y_pred_pol)
r2_pol = r2_score(y_test_pol, y_pred_pol)

print(y_pred_pol)
print("ERRORES PRUEBA COMBINADA")
print(["RMSE:", rmse_pol], ["MAE", mae_pol], ["R2:", r2_pol])

C:\Users\PC\IdeaProjects\python\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.439e+05, tolerance: 4.089e+04
  model = cd_fast.enet_coordinate_descent(


mejor parametro polinomial:  {'lasso__alpha': 1, 'polynomialfeatures__degree': 3}
[ 52.17784135 184.90942767 257.67710621 ... 242.50095592 236.83229257
 215.08341322]
ERRORES PRUEBA COMBINADA
['RMSE:', np.float64(130.87517332192868)] ['MAE', np.float64(94.6369852138807)] ['R2:', 0.4904711357476301]


In [29]:
modelo_lasso= mejor_modelo_pol.named_steps['lasso']
coeficientes = modelo_lasso.coef_
nombres_caracteristicas = mejor_modelo_pol.named_steps['polynomialfeatures'].get_feature_names_out(x_train_pol.columns)
contador = 0
for nombre, coef in zip(nombres_caracteristicas, coeficientes):
    if abs(coef) != 0:
        print(f"{nombre}: {coef}")
        contador = contador +1
print(contador)


temp^2: -0.5753990527907971
temp atemp: -4.422285121745846
temp hum: -0.3244065630388106
atemp weathersit_Clear: 1.899465215590302
hum^2: -6.2438062884986865
hum season_Winter: 3.1070977524687304
hum weathersit_Mist: 5.854660681282516
windspeed^2: -2.2030530135872493
windspeed time_of_day_Night: 0.5783531545956268
temp^2 hum: 3.656006730459068
temp^2 weathersit_Clear: -0.00034127993096216044
temp^2 time_of_day_Evening: -6.17678364286886
temp^2 time_of_day_Night: 9.217714627493285
temp atemp hum: 0.9641136876043058
temp hum^2: -1.8243504552683572
temp hum windspeed: 0.2409510917255057
temp hum weekday_0: -4.239715498406497
temp hum weekday_2: 0.650690966445555
temp hum season_Spring: -1.846530390771873
temp hum season_Summer: 2.1375461136503664
temp hum weathersit_Mist: -2.087546813358916
temp windspeed weekday_6: 3.9637749476202453
temp weekday_0^2: 4.560139848648079
temp weekday_2 weathersit_Clear: -1.089181516558267
temp weekday_2 weathersit_Light Rain: 0.3542827739969972
temp weekda